In [78]:
!cat './data/id=10_n=5_p=0.5_k=3.cnf'

p cnf 15 41
1 2 3 0
4 5 6 0
7 8 9 0
10 11 12 0
13 14 15 0
-1 -2 0
-1 -3 0
-2 -3 0
-4 -5 0
-4 -6 0
-5 -6 0
-7 -8 0
-7 -9 0
-8 -9 0
-10 -11 0
-10 -12 0
-11 -12 0
-13 -14 0
-13 -15 0
-14 -15 0
-1 -4 0
-2 -5 0
-3 -6 0
-1 -10 0
-2 -11 0
-3 -12 0
-4 -7 0
-5 -8 0
-6 -9 0
-4 -10 0
-5 -11 0
-6 -12 0
-4 -13 0
-5 -14 0
-6 -15 0
-7 -10 0
-8 -11 0
-9 -12 0
-7 -13 0
-8 -14 0
-9 -15 0


In [75]:
import os
import pdb
import random
from collections import namedtuple

import scipy.sparse as sparse
import torch

from cnf import CNF
from util import DataSample, adj, adj_batch, init_edge_attr, to_sparse_tensor


Batch = namedtuple('Batch', ['x', 'adj', 'sol'])
SampleWrapper = namedtuple('SampleWrapper', ['ptype', 'sample'])


def load_dir(path):
    data = []
    for filename in os.listdir(path):
        name, ext = os.path.splitext(filename)
        if ext != '.cnf':
            continue
        f = CNF.from_file(os.path.join(path, filename))
        if name.startswith('uu'):
            continue
        data.append(DataSample(filename, f, adj(f), None))
    return data


def load_dir_wrapper(path):
    data = load_dir(path)
    return [SampleWrapper(ptype, sample) for sample in data]


def init_tensor_wrapper(sample_wrapper, device):
    return (sample_wrapper.ptype, init_tensors(sample_wrapper.sample, device))

In [3]:
import numpy as np

In [29]:
data = load_dir('../data/kcolor/3-5-0.5/')

In [5]:

def adj_sign(n, m, occur):
    i = np.repeat(range(n), [len(lst) for lst in occur])
    j = np.concatenate(occur)
    v = np.ones(len(i), dtype=np.int64)
    return sparse.coo_matrix((v, (i, j)), shape=(n, m))


def adj(f):
    n, m, occur = f.n_variables, len(f.clauses), f.occur_list
    adj_pos = adj_sign(n, m, occur[1 : n + 1])
    adj_neg = adj_sign(n, m, occur[:n:-1])
    return (adj_pos, adj_neg)


def adj_batch(adjs, fstack):
    adjp, adjn = list(zip(*adjs))
    return fstack((sparse.block_diag(adjp), sparse.block_diag(adjn)))


def to_sparse_tensor(x):
    x = x.tocoo()
    i = torch.tensor(np.vstack((x.row, x.col)), dtype=torch.int64)
    v = torch.tensor(x.data, dtype=torch.float32)
    return torch.sparse_coo_tensor(i, v, torch.Size(x.shape))


def init_edge_attr(k):
    return torch.cat(
        (
            torch.tensor([1, 0], dtype=torch.float32).expand(k, 2),
            torch.tensor([0, 1], dtype=torch.float32).expand(k, 2),
        ),
        dim=0,
    )


def normalize(x):
    return 2 * x - 1


def unnormalize(x):
    return (x + 1) / 2


In [9]:
data.formula.n_variables, len(data.formula.clauses)

(15, 41)

In [59]:
CNF.from_file?

Signature: CNF.from_file(filename)
Docstring: <no docstring>
File:      ~/Code/class/research/sat/code/cnf.py
Type:      method

In [31]:
def init_tensors(sample, device):
    # [1,0,0] -> assigned False
    # [0,1,0] -> assigned True
    # [0,0,1] -> clause
    adj = sample.adj
    n, m = adj[0].shape[0], adj[0].shape[1]
    xv = torch.zeros(n, 3, dtype=torch.float32)
    sol = [x if random.random() < 0.5 else -x for x in range(n + 1)]
    xv[torch.arange(n), (torch.tensor(sol[1:]) > 0).long()] = 1
    xv = xv.to(device)
    xc = torch.tensor([0, 0, 1], dtype=torch.float32).repeat(m, 1).to(device)
    xev = init_edge_attr(n).to(device)
    xec = init_edge_attr(m).to(device)
    vadj = to_sparse_tensor(sparse.hstack(adj)).to(device)
    cadj = to_sparse_tensor(sparse.vstack(adj)).t().to(device)
    return Batch((xv, xc, xev, xec), (vadj, cadj), sol)


In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [33]:
tensor = init_tensors(data[49], device)

In [79]:
data[49].adj

(<15x32 sparse matrix of type '<class 'numpy.int64'>'
 	with 15 stored elements in COOrdinate format>,
 <15x32 sparse matrix of type '<class 'numpy.int64'>'
 	with 54 stored elements in COOrdinate format>)

In [95]:
tensor.adj

(tensor(indices=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                         14,  0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  3,
                          4,  4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,
                          7,  7,  7,  7,  8,  8,  8,  8,  8,  9,  9,  9,  9, 10,
                         10, 10, 10, 11, 11, 11, 11, 12, 12, 13, 13, 14, 14],
                        [ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,
                          4, 37, 38, 52, 37, 39, 53, 38, 39, 54, 40, 41, 55, 58,
                         40, 42, 56, 59, 41, 42, 57, 60, 43, 44, 52, 55, 61, 43,
                         45, 53, 56, 62, 44, 45, 54, 57, 63, 46, 47, 58, 61, 46,
                         48, 59, 62, 47, 48, 60, 63, 49, 50, 49, 51, 50, 51]]),
        values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                       1., 1., 1., 1

In [35]:
tensor.adj[0].shape  #clauses

torch.Size([15, 64])

In [ ]:
tensor.x[2].shape  #edge 1

In [ ]:
tensor.x[3].shape  #edge 2

In [60]:
tensor[:2]

((tensor([[1., 0., 0.],
          [1., 0., 0.],
          [1., 0., 0.],
          [1., 0., 0.],
          [0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.],
          [0., 1., 0.],
          [0., 1., 0.],
          [0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.],
          [0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.]]),
  tensor([[0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0.,

In [72]:
data[0].filename

'id=103_n=5_p=0.5_k=3.cnf'